In [ ]:
import os
import time
import shutil
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist,cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3

from scipy.linalg import sqrtm
from skimage.transform import resize
import matplotlib.pyplot as plt
import keras.backend as K
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from keras.layers import Conv2D, Conv2DTranspose, Reshape, Dropout, Flatten
from keras.optimizers.legacy import Adam
import warnings
warnings.filterwarnings('ignore')

In [ ]:
INCEPTION_SHAPE=(299,299,3)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

inception = InceptionV3(include_top=False, pooling='avg', input_shape=INCEPTION_SHAPE)

def scale_images(images, new_shape):
    """
    Scales images into new shape by interpolation since inception model requires that
    :param images: List of images
    :param new_shape: The transformed shape
    :return: Transformed images
    """

    images_list = list()

    for image in images:
        new_image = resize(image, new_shape)
        images_list.append(new_image)

    return np.asarray(images_list)


def calculate_fid(model, images1, images2):
    act1 = model.predict(images1)
    act2 = model.predict(images2)

    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    ssdiff = np.sum((mu1 - mu2) ** 2.0)

    covmean = sqrtm(sigma1.dot(sigma2))

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # Equation for FID score calculation
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


"""def calculate_is(images, n_split=10, eps=1E-16):
 # load inception v3 model
    model = inception
    # convert from uint8 to float32
    processed = images.astype('float32')
    # pre-process raw images for inception v3 model
    processed = preprocess_input(processed)
    # predict class probabilities for images
    yhat = model.predict(processed)
    # enumerate splits of images/predictions
    scores = list()
    n_part = np.floor(images.shape[0] / n_split)
    for i in range(n_split):
    # retrieve p(y|x)
        ix_start, ix_end = int(i * n_part), int(i * n_part + n_part)
        p_yx = yhat[ix_start:ix_end]
        # calculate p(y)
        p_y = np.expand_dims(p_yx.mean(axis=0), 0)
        # calculate KL divergence using log probabilities
        kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
        # sum over classes
        sum_kl_d = kl_d.sum(axis=1)
        # average over images
        avg_kl_d = np.mean(sum_kl_d)
        # undo the log
        is_score = np.exp(avg_kl_d)
        # store
        scores.append(is_score)
        # average across images
        is_avg, is_std = np.mean(scores), np.std(scores)
    return is_avg, is_std"""

1 Physical GPUs, 1 Logical GPU


"def calculate_is(images, n_split=10, eps=1E-16):\n # load inception v3 model\n    model = inception\n    # convert from uint8 to float32\n    processed = images.astype('float32')\n    # pre-process raw images for inception v3 model\n    processed = preprocess_input(processed)\n    # predict class probabilities for images\n    yhat = model.predict(processed)\n    # enumerate splits of images/predictions\n    scores = list()\n    n_part = np.floor(images.shape[0] / n_split)\n    for i in range(n_split):\n    # retrieve p(y|x)\n        ix_start, ix_end = int(i * n_part), int(i * n_part + n_part)\n        p_yx = yhat[ix_start:ix_end]\n        # calculate p(y)\n        p_y = np.expand_dims(p_yx.mean(axis=0), 0)\n        # calculate KL divergence using log probabilities\n        kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))\n        # sum over classes\n        sum_kl_d = kl_d.sum(axis=1)\n        # average over images\n        avg_kl_d = np.mean(sum_kl_d)\n        # undo the log\n 

DCGAN

In [ ]:

class DCGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = True

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            new_gen_imgs=scale_images(gen_imgs,(75,75,3))
            new_imgs=scale_images(imgs,(75,75,3))
            #inception_score=calculate_inception_score(new_gen_imgs)
            #fid_score=calculate_fid(inception,new_gen_imgs, new_imgs)
            
            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            #print(f"FID Score: {fid_score},  Inception Score: {inception_score}")
            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
        self.generator.save("/content/drive/MyDrive/DLPROJE/model/gen/")
        self.discriminator.save("/content/drive/MyDrive/DLPROJE/model/disc/")
        self.combined.save("/content/drive/MyDrive/DLPROJE/model/gan")
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/drive/MyDrive/DLPROJE/dcganmnist/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = DCGAN()
    gan.train(epochs=10000, batch_size=256, sample_interval=50)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
7524 [D loss: 0.752753, acc.: 50.00%] [G loss: 0.411508]
8/8 [==============================] - 0s 3ms/step
7525 [D loss: 0.749926, acc.: 50.00%] [G loss: 0.412624]
8/8 [==============================] - 0s 3ms/step
7526 [D loss: 0.749610, acc.: 50.00%] [G loss: 0.410742]
8/8 [==============================] - 0s 3ms/step
7527 [D loss: 0.750056, acc.: 50.00%] [G loss: 0.411659]
8/8 [==============================] - 0s 3ms/step
7528 [D loss: 0.756733, acc.: 50.00%] [G loss: 0.410421]
8/8 [==============================] - 0s 3ms/step
7529 [D loss: 0.753017, acc.: 50.00%] [G loss: 0.415520]
8/8 [==============================] - 0s 2ms/step
7530 [D loss: 0.752846, acc.: 50.00%] [G loss: 0.411541]
8/8 [==============================] - 0s 3ms/step
7531 [D loss: 0.751234, acc.: 50.00%] [G loss: 0.414507]
8/8 [==============================] - 0s 3ms/step
7532 [D loss: 0.749369, acc.: 50.00%] [G loss: 0.414128]
8/8 [===========================

9999 [D loss: 0.752107, acc.: 50.00%] [G loss: 0.410264]
